### This script parses the raw XML files. It is used to create the TXT files and an intermediate annotation CSV file that includes the tags made by the Ibsen center.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import glob

pd.set_option('display.max_colwidth', None)

Testing the different functionalities of Beautifoulsoup4

In [2]:
xml_doc = """ <?xml version="1.0" encoding="UTF-8"?><text xmlns="http://www.tei-c.org/ns/1.0" xmlns:HIS="http://www.example.org/ns/HIS" rend="letter" corresp="B18630810CP" xml:lang="da-no">
<body>
<div type="letter">
<dateline>
                            <pb n="[1]"/>Christiania <date>10/8. 63</date>.</dateline>
<salute>Hr: Clemens Petersen.</salute>
<p>Jeg kan aldrig <anchor type="lemma" xml:id="koB12_797"/>komme af med Brevskrivning, navnlig fordi jeg har en Mistanke om at man paa mit Forfatterskab i denne Retning med adskillig Grund kan anvende <anchor type="lemma" xml:id="koB12_798"/>den Charakteristik, hvormed De, som det forekommer mig temmelig haardt, betegner min Prosa ialmindelighed; <HIS:hisRef type="tcNote" xml:id="noteT12_196" target="B1844-1871ht_noter.xml" corresp="noteT12_196">alligevel</HIS:hisRef> maa jeg dog tilskrive Dem etpar Linjer for oprigtigt og hjerteligt at takke for <anchor type="lemma" xml:id="koB12_799"/>Deres Anmeldelse af min Bog. Jeg takker Dem baade for hvad jeg kan være enig med Dem i (hvilket ikke blot og bart er de for mig smigrende Partier af Kritiken) og for hvad jeg, naar jeg engang bliver saa heldig personlig at træffe Dem, ialfald vil <emph>forsøge</emph> at disputere om. Navnlig takker jeg Dem fordi jeg seer at De dog igrunden ikke har saameget imod mig, som jeg hidindtil instinktmæssig har indbildt mig; dette har for mig en Vægt, som maaske ikke vil være let at gjøre Dem indlysende, der ikke kjender til i hvilken forfærdelig Grad jeg gaar sjæleligt allene heroppe. Mine «Venners» <pb n="[2]"/>Opfatning af mig gjør mig forresten ingen Skade; jeg seer, angaaende mig selv, i ethvert Punkt klarere end alle mine Venner – og dette sandelig ikke til min Fordel. – <anchor type="lemma" xml:id="koB12_800"/>Jeg skriver nu paa <HIS:hisRef type="work" target="Navneregister_HISe.xml#woKE">et historisk Skuespil i 5 Akter</HIS:hisRef>, – men i Prosa, jeg <emph>kan</emph> ikke skrive det i Vers. – De gjør mig lidt Uret naar De antyder at jeg har forsøgt paa at ville ligne <HIS:hisRef type="person" target="Navneregister_HISe.xml#peBB">Bjørnson</HIS:hisRef>; <HIS:hisRef type="work" target="Navneregister_HISe.xml#woF1">
                                <anchor type="lemma" xml:id="koB12_801"/>«Fru Inger til Østeraad»</HIS:hisRef> og <HIS:hisRef type="work" target="Navneregister_HISe.xml#woHH">
                                <anchor type="lemma" xml:id="koB12_802"/>«Hærmændene paa Helgeland»</HIS:hisRef> er skrevne <anchor type="lemma" xml:id="koB12_803"/>forinden B: endnu havde digtet en Linje. (NB: det er muligt at «Mellem Slagene» var til paa den Tid jeg skrev «Hærmændene» men kom ikke og kunde ikke komme mig for Øje). – Hvad <HIS:hisRef type="work" target="Navneregister_HISe.xml#woKK">
                                <anchor type="lemma" xml:id="koB12_804"/>«Kjærlighedens Komedie»</HIS:hisRef> angaar, saa kan jeg forsikkre Dem, at har det nogensinde været en Nødvendighed for en Forfatter at skille sig af med en Stemning og et Stof, saa var det Tilfældet <anchor type="lemma" xml:id="koB12_805"/>da jeg gik ivej med dette Arbejde. – <anchor type="lemma" xml:id="koB12_806"/>Deres venlige Raad at indsende «Fru Inger» til <HIS:hisRef type="org" target="Navneregister_HISe.xml#orgKT">det kgl: Theater</HIS:hisRef> skal jeg følge; gid jeg blot maatte gribe Tingen paa den rette Maade og at den maatte lykkes. Disse Linjer til Tak har det været mig en Trang at sende Dem idet jeg varmt og levende føler at De har gjort en god Gjerning mod mig ved ikke at lægge min Bog tilside i Taushed.</p>
<salute>Deres forbundne</salute>
<signed>Henr: Ibsen</signed>
</div>
</body>
</text> """

In [3]:
soup = BeautifulSoup(xml_doc, 'xml')

In [4]:
h = soup.find_all(attrs={"type": "work"})
for i in h:
    n = i.get_text()
    print(n)

et historisk Skuespil i 5 Akter

«Fru Inger til Østeraad»

«Hærmændene paa Helgeland»

«Kjærlighedens Komedie»


In [5]:
soup.find_all(attrs={"type": "org"})

[<HIS:hisRef target="Navneregister_HISe.xml#orgKT" type="org">det kgl: Theater</HIS:hisRef>]

In [18]:
soup.find_all(["p"])

[<p>Jeg kan aldrig <anchor type="lemma" xml:id="koB12_797"/>komme af med Brevskrivning, navnlig fordi jeg har en Mistanke om at man paa mit Forfatterskab i denne Retning med adskillig Grund kan anvende <anchor type="lemma" xml:id="koB12_798"/>den Charakteristik, hvormed De, som det forekommer mig temmelig haardt, betegner min Prosa ialmindelighed; <HIS:hisRef corresp="noteT12_196" target="B1844-1871ht_noter.xml" type="tcNote" xml:id="noteT12_196">alligevel</HIS:hisRef> maa jeg dog tilskrive Dem etpar Linjer for oprigtigt og hjerteligt at takke for <anchor type="lemma" xml:id="koB12_799"/>Deres Anmeldelse af min Bog. Jeg takker Dem baade for hvad jeg kan være enig med Dem i (hvilket ikke blot og bart er de for mig smigrende Partier af Kritiken) og for hvad jeg, naar jeg engang bliver saa heldig personlig at træffe Dem, ialfald vil <emph>forsøge</emph> at disputere om. Navnlig takker jeg Dem fordi jeg seer at De dog igrunden ikke har saameget imod mig, som jeg hidindtil instinktmæssig ha

Gathering all the letters and parsing the p tags to the text files.

In [ ]:
file_list = glob.glob(os.path.join(os.getcwd(), r"C:\\Users\\Sarah\\Desktop\Bachelor\\Ibsen\\data_ibsen_work\\xml", "BREV*.xml"))
path = (r"C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\data_ibsen_work\\txt\\brev_p\\") 

for file in file_list:
    with open(file) as f_input:
        corpus = ''
        soup = BeautifulSoup(f_input, 'xml')
        all_p = soup.find_all(["p"])
        for tag in all_p:
            corpus += tag.get_text().strip().replace("\n", "") 
        file = (''.join([n for n in os.path.basename(file_path)]))
        file = re.sub('\.xml', '', file) 
        text_file = open(path + F"{file}.txt", "w")
        n = text_file.write(corpus)
        text_file.close()

Gathering all the letters and creating the annotation CSV files while keeping the Ibsen tags of the relevant categories

In [150]:
file_list = glob.glob(os.path.join(os.getcwd(), r"C:\\Users\\Sarah\\Desktop\Bachelor\\Ibsen\\data_ibsen_work\\xml", "BREV*.xml"))
path = (r"C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\data_ibsen_work\\txt\\brev_p\\")

strings = []
files = []
corpus_all = []
person = []
place = []
org = []
work = []

for file in file_list:
    with open(file, encoding = 'utf-8') as f_input:
        corpus = ''
        
        soup = BeautifulSoup(f_input, 'xml')
        all_p = soup.find_all(["p"])
        for tag in all_p:
            corpus += tag.get_text().strip().replace("\n", "")
        corpus_all.append(corpus)
        
        person_in = []
        person_tags = soup.find_all(attrs={"type": "person"})
        for element in person_tags:
            person_text = element.get_text().strip().replace("\n", "")
            person_in.append(person_text)
        person.append(person_in)
        
        place_in = []
        place_tags = soup.find_all(attrs={"type": "place"})
        for element in place_tags:
            place_text = element.get_text().strip().replace("\n", "")
            place_in.append(place_text)
        place.append(place_in)

        
        org_in = []
        org_tags = soup.find_all(attrs={"type": "org"})
        for element in org_tags:
            org_text = element.get_text().strip().replace("\n", "")
            org_in.append(org_text)
        org.append(org_in)
        
        work_in = []
        work_tags = soup.find_all(attrs={"type": "work"})
        for element in work_tags:
            work_text = element.get_text().strip().replace("\n", "")
            work_in.append(work_text)
        work.append(work_in)
        
        
        file_name = (''.join([n for n in os.path.basename(file)]))
        file_name = re.sub('\.xml', '', file_name) 
        files.append(file_name)

In [151]:
lists = {'files': files, 'text': corpus_all, 'person': person, 'place': place, 'org': org, 'work': work}

df = pd.DataFrame.from_dict(lists, orient = 'index')
df = df.transpose()

In [6]:
path = (r"C:\\Users\\Sarah\\Desktop\\Bachelor\\Ibsen\\data_ibsen_work\\csv\\")
df.to_csv(os.path.join(path,r'brev_for_annotations.csv'), encoding = 'utf-8', index = False)

NameError: name 'df' is not defined

In [155]:
df

,files,text,person,place,org,work
0,BREV_B1844-1871ht_18670308FH,,[],[],[],[]
1,BREV_B1844-1871ht_B18260306NTB,,[],[],[],[]
2,BREV_B1844-1871ht_B18440520PL,"Du maa virkelig undskylde, at jeg først nu besvarer Dit Brev, men her har været saameget at bestille i denne Tid, at det har været mig umuligt før, og af Mangel paa Tid kan jeg heller ikke idag skrive noget langt Brev. – Hedevall er vel allerede reist, og han vil vist blive fornøiet, idetmindste er jeg meget vel tilfreds og har aldrig angret at jeg er kommen hid, da Reimann er meget god imod mig, og gjør alt muligt, for at opvække Lyst hos mig til Apotheket, da denne i Begyndelsen ikke var synderlig stor. Hans Kone derimod gjør jeg langtfra ikke saameget af, og vi ere ofte Uvenner, da det er umuligt at gjøre hende Noget tilpas. Reimann er ogsaa Postmester, og Du kan derfor gjerne lade min Broder, Johan, lægge Dine Breve ind i dem han skriver, da Du paa den Maade undgaaer at betale Noget for dem. Du kan tro at Grimstad, og især Omegnen er ganske vakker; og Damerne, endskjøndt ikke saa galante, som Skiens, ere ogsaa ganske antagelige, og Du kan være overbeviist om, at jeg gjør alt for at erhverve mig deres Naade, som ogsaa er temmelig let at erholde. Da Dampskibet passerer Grimstad to Gange ugentlig, haaber jeg, at gjøre en Tour med det til Skien, dersom ellers ingen Hindringer kommer iveien, som jeg dog ikke skulle formode. Jeg har adskillige Spørgsmaale at gjøre Dig, som du maa besvare med næste Post: Først og fremst maa Du sige mig hvorledes J: J: optog Efterretningen om hendes Kjærestes Død, og lad mig tillige vide, hvem den Lykkelige er, som har indtaget hans Plads; thi jeg kjender hende for godt til at antage at hun endnu sørger for ham. Dernest maa Du sige mig om Carl Aamodt endnu øver sig i Poesien og hvis dette er Tilfældet maa Du endelig bede ham ikke at forglemme, at sende mig et lille Digt, for at jeg kan see om han har gjordt nogen Fremgang. – Uagtet jeg nok kunde have mere at skrive om, maa jeg dog afbryde af Mangel paa Tid, men Du kan være overbeviist om, næste Gang at faa et længere Brev. – Vær saa god at sende Bogen «Wilhelm Tell» som Hedevall har laant, optill os, da den ikke tilhører mig selv. – Lev vel og hils alle gode Venner fra Din hengivneLad endelig ikke Nogen see dette da det er skrevet i største Hast.","[Hedevall, Reimann, Hans Kone, Johan, J: J:, Carl Aamodt]","[Grimstad, Skiens]",[],[]
3,BREV_B1844-1871ht_B18441006HSte,"Tilgiv at jeg ikke før har besvaret Dit Brev, men det har været mig reent umuligt. I hele Sommer har jeg glædet mig til, at jeg skulde komme hjem til Din Confirmation; men det kan ikke blive noget af, da her fortiden er meget at bestille, og Reimann har just ikke synderlig Lyst til at gjøre noget selv; jeg kan saaledes ikke mundtlig aflægge Dig min Lykønskning til denne Dag; hvortil vistnok ogsaa Du ligesaavel som de fleste andre Smaapiger længes efter, for det er jo saa moersomt at være voxen – idetmindste synes man saaledes her. Iforgaars afbrød en vigtig Begivenhed det ellers saa eensformige Liv her – idet nemmelig en af Byens respectable Handværkere havde Bryllup. Det var ret en Festdag for Mdm Reimann endskjøndt hun rigtignok ikke var med, – et Bryllup er altid en interessant Ting for Damerne, ved Du nok – og naturligviis ogsaa for Mdm R... hun vidste vidt og bredt at fortælle alt hvad man havde talt, gjort og spiist, og det er en sand Fornøielse at høre hende fortælle Noget, da man derved faaer at vide langt mere, end om man selv havde seet det. Forøvrigt veed jeg intet at fortælle Dig, undtagen den høist sørgelige Begivenhed, at Løverdagen den 5te Dennes afgik ved Døden – Reimanns Yndlingskat efter faae Timer iforveien at have født til Verden 2 trivelige Unger, der, skjønt endnu for smaa til fuldkommen at fatte deres Tab, tilligemed Reimann beklage Tabet af en øm Moder og kjærlig Veninde – Den kjære Afdøde, hvis gode Forstand, tilligemed flere Legemsfortrin, udmærkede den fremfor de fleste af dens Lige, var ogsaa en god Mus